In [8]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForMaskedLM, BertweetTokenizer

config = AutoConfig.from_pretrained('vinai/bertweet-base')
tokenizer = AutoTokenizer.from_pretrained('vinai/bertweet-base')
model = AutoModelForMaskedLM.from_config(config)

Downloading: 100%|██████████| 558/558 [00:00<00:00, 559kB/s]
Downloading: 100%|██████████| 824k/824k [00:01<00:00, 598kB/s] 
Downloading: 100%|██████████| 1.03M/1.03M [00:01<00:00, 674kB/s] 
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
from glob import glob
from datasets import Dataset

import re
import pandas as pd
def mask_data_loading(url):
    def stock_symbol_mask(sentense):
        pattern = r'\$[A-Z]*'
        # symbol += re.findall(pattern, sentense)
        result = re.sub(pattern, tokenizer.mask_token, sentense)

        return result

    with open(url, 'r', encoding='utf-8') as f:
        df = pd.read_json(f)
        data = df.copy()
        data = data.loc[df['sentiment'].notnull()]
        data['sentiment'] = pd.Categorical(data['sentiment'])
        data['sentense'] = data['body'].map(stock_symbol_mask)
        data['labels'] = data['body']
        symbols = set()
        for symbol_list in data['body'].str.findall(r'\$[A-Z]+'):
            for symbol in symbol_list:
                symbols.add(symbol)
        return data, symbols

data_url = '../crawler/stock/data/**.json'
url = glob(data_url)[-1]
data, symbols = mask_data_loading(url)

dataset = Dataset.from_pandas(data.loc[:, ['labels', 'sentense']])
dataset = dataset.remove_columns('__index_level_0__')

special_tokens_dict = {'additional_special_tokens': list(symbols)}
tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(29625, 768)

In [7]:
tokenizer.save_pretrained('./symbol-vocab')

('./symbol-vocab\\tokenizer_config.json',
 './symbol-vocab\\special_tokens_map.json',
 './symbol-vocab\\vocab.txt',
 './symbol-vocab\\added_tokens.json',
 './symbol-vocab\\tokenizer.json')

In [3]:
def encode(example):
    label = tokenizer(example['labels'], padding='max_length', truncation=True)
    # 101, 51, 1234, 12541, 151
    result = tokenizer(example['sentense'], padding='max_length', truncation=True)
    # 101, 103, 103, 103
    result['label_ids'] = label['input_ids']

    # masked_position = [i for i in range(len(result['input_ids'])) if result['input_ids'][i] == tokenizer.mask_token_id]
    # result['decoder_input_ids'] = label['input_ids']
    # result['labels'] = [-100 for i in label['input_ids']]
    # for i in range(len(result['labels'])):
    #     if not i in masked_position:
    #         result['labels'][i] = -100

    return result

encoded_dataset = dataset.map(encode, batched=True)

# print(encoded_dataset[0]['sentense'])
# print(encoded_dataset[0]['input_ids'])
# print(encoded_dataset[0]['labels'])
# print(encoded_dataset[0]['label_ids'])
# print(tokenizer.ids_to_tokens)
# print(tokenizer.decode(encoded_dataset[0]['label_ids']))

100%|██████████| 15/15 [00:05<00:00,  2.86ba/s]


In [4]:
from transformers import DataCollatorForLanguageModeling

# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

# samples = encoded_dataset[:2]
# for chunk in data_collator(samples)["input_ids"]:
#     print(f"\n'>>> {tokenizer.decode(chunk)}'")

# Training

In [5]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [6]:
from transformers import TrainingArguments, Trainer, Seq2SeqTrainingArguments, Seq2SeqTrainer

# training_args = TrainingArguments(
#     output_dir="test_trainer",
#     per_device_train_batch_size=2,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=encoded_dataset,
#     compute_metrics=compute_metrics,
# )

# Seq2SeqTrainer
training_args = Seq2SeqTrainingArguments(
    output_dir="test_trainer",
    per_device_train_batch_size=4,
    num_train_epochs=50,
    save_steps=10000
)

train_datset = encoded_dataset.shuffle().select(range(4000))

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_datset,
    compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: sentense. If sentense are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
C:\Users\douli\anaconda3\envs\stock-nlp\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4000
  Num Epochs = 50
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 50000
  1%|          | 500/50000 [02:55<4:47:32,  2.87it/s]

{'loss': 1.7203, 'learning_rate': 4.9500000000000004e-05, 'epoch': 0.5}


  2%|▏         | 1000/50000 [05:48<4:27:32,  3.05it/s]

{'loss': 0.0397, 'learning_rate': 4.9e-05, 'epoch': 1.0}


  3%|▎         | 1500/50000 [08:32<4:25:09,  3.05it/s]

...

100%|██████████| 50000/50000 [4:34:04<00:00,  3.05it/s]Saving model checkpoint to test_trainer\checkpoint-50000
Configuration saved in test_trainer\checkpoint-50000\config.json


{'loss': 0.0001, 'learning_rate': 0.0, 'epoch': 50.0}


Model weights saved in test_trainer\checkpoint-50000\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 50000/50000 [4:34:09<00:00,  3.04it/s]

{'train_runtime': 16449.4913, 'train_samples_per_second': 12.158, 'train_steps_per_second': 3.04, 'train_loss': 0.019213099329695105, 'epoch': 50.0}


TrainOutput(global_step=50000, training_loss=0.019213099329695105, metrics={'train_runtime': 16449.4913, 'train_samples_per_second': 12.158, 'train_steps_per_second': 3.04, 'train_loss': 0.019213099329695105, 'epoch': 50.0})

In [7]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer,
    device=0
)

s = '[MASK] to the sky!!!'
fill_mask(s)

[{'score': 0.9999961853027344,
  'token': 29304,
  'token_str': '$TSLA',
  'sequence': 'to the sky!!!'},
 {'score': 2.9771953791168926e-07,
  'token': 29623,
  'token_str': '$TSM',
  'sequence': 'to the sky!!!'},
 {'score': 2.534041811941279e-07,
  'token': 29364,
  'token_str': '$MSFT',
  'sequence': 'to the sky!!!'},
 {'score': 2.319639662573536e-07,
  'token': 29125,
  'token_str': '$FB',
  'sequence': 'to the sky!!!'},
 {'score': 1.744811584103445e-07,
  'token': 29180,
  'token_str': '$MULN',
  'sequence': 'to the sky!!!'}]

In [8]:
import torch
torch.cuda.empty_cache()
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |    1673 MB |    4893 MB |  613870 GB |  613869 GB |
|       from large pool |    1671 MB |    4890 MB |  613778 GB |  613777 GB |
|       from small pool |       2 MB |       2 MB |      91 GB |      91 GB |
|---------------------------------------------------------------------------|
| Active memory         |    1673 MB |    4893 MB |  613870 GB |  613869 GB |
|       from large pool |    1671 MB |    4890 MB |  613778 GB |